In [ ]:
from google.colab import drive
import os, sys
drive.mount('/content/drive/')

In [ ]:
!pip install cdsapi netCDF4

# SST

In [ ]:
import netCDF4 as nc
import pandas as pd
import datetime

# read
path = '/content/drive/MyDrive/master_thesis_data/sst.nc'
ds = nc.Dataset(path)

# get variables
time = ds['TIME'][:].data
nat = ds['NAT'][:].data

# create df
nat_df = pd.DataFrame({'time':time, 'nat':nat})

# create date column based on fact that time is the distance in days from 1800-01-01
start_date = datetime.date(1800, 1, 1)
nat_df['Date'] = nat_df['time'].apply(lambda x: start_date + datetime.timedelta(days=x))
nat_df['Date'] = pd.to_datetime(nat_df['Date'])

nat_df.drop(columns=['time'], inplace=True)

# sst df is missing dates, create range of all dates and merge this to the df
min_date, max_date = nat_df['Date'].min(), nat_df['Date'].max()
date_range = pd.date_range(start=min_date, end=max_date, freq='D')
df_dates = pd.DataFrame(date_range, columns=['Date'])
df_dates['Date'] = pd.to_datetime(df_dates['Date'])

nat_df_all_dates = df_dates.merge(nat_df, on = 'Date', how = 'left')

# back fill the empty days
nat_df_filled = nat_df_all_dates.fillna(method='bfill')

nat_df_filled.to_csv('/content/drive/MyDrive/master_thesis_data/processed_data/sst_processed.csv')

# NAO

In [ ]:
import pandas as pd


def preprocess(path: str) -> pd.DataFrame:
    """
    Reset index, index is now the year and we want this as a column.
    Melt the dataset into a year, month and NAO column.
    Rename index to Year.
    Map the months (now strings) to corresponding month numbers.
    """
    nao = pd.read_csv(path, delim_whitespace=True, index_col=0)
    nao.reset_index(inplace=True)
    nao_melted = pd.melt(nao, id_vars=['index'], var_name='Month', value_name='NAO')
    nao_melted.rename(columns={'index':'Year'}, inplace=True)
    month_mapping = {'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 'May': 5, 'Jun': 6,
                 'Jul': 7, 'Aug': 8, 'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12}
    nao_melted['Month'] = nao_melted['Month'].map(month_mapping)

    return nao_melted

nao_path = '/content/drive/MyDrive/master_thesis_data/nao.txt'
nao = preprocess(nao_path)

nao

In [ ]:
nao.to_csv('/content/drive/MyDrive/master_thesis_data/processed_data/nao_processed.csv')

# Weather

In [ ]:
import pandas as pd

def read_data(path: str) -> pd.DataFrame:
    """
    Read the data.
    Strip spaces from the columns.
    Rename all the columns (abbreviations -> full names).
    """

    weather = pd.read_csv(path)

    weather.columns = weather.columns.str.strip()
    weather = weather.applymap(lambda x: x.strip() if isinstance(x, str) else x)
    weather.replace('', None, inplace=True)

    column_mapping = {
        'YYYYMMDD': 'Date',
        'RH': 'dailyPrecipitation',
        'TG': 'dailyMeanTemperature',
        'FG': 'dailyMeanWindspeed',
    }

    weather = weather[column_mapping.keys()]
    weather.rename(columns=column_mapping, inplace=True)

    return weather


def filter_on_year(weather: pd.DataFrame) -> pd.DataFrame:
    """
    Filter on only 2021 and later and reset index after filtering.
    """
    weather = weather[weather['Date'] >= 20060101]
    weather.reset_index(drop=True, inplace=True)

    return weather


def cast_datekey(weather: pd.DataFrame) -> pd.DataFrame:
    """
    Cast datekey 20210101 to format 2021/01/01
    """
    weather['Date'] = pd.to_datetime(weather['Date'], format='%Y%m%d').dt.strftime('%Y-%m-%d')

    return weather


def convert_columns(weather: pd.DataFrame) -> pd.DataFrame:
    """
    Divide the following columns by 10 to get actual number, since they are 0.1 of the actual number.
    """
    cols_to_convert = [
        'dailyPrecipitation',
        'dailyMeanTemperature',
        'dailyMeanWindspeed',
    ]
    for i in cols_to_convert:
        weather[i] = weather[i] / 10

    return weather


def create_date_columns(weather: pd.DataFrame) -> pd.DataFrame:
    """
    Cast date column (2021-01-01 instead of 2021/01/01) and add month and year columns
    Create a week key combining year and week number
    """
    weather['Date'] = pd.to_datetime(weather['Date'], format='%Y-%m-%d')
    weather['Month'] = weather['Date'].dt.month

    return weather


def preprocess(path: str) -> pd.DataFrame:
    """
    Preprocess the weather data.
    """
    weather = read_data(path)
    weather = filter_on_year(weather)
    weather = cast_datekey(weather)
    # weather = cast_hour_columns(weather)
    weather = convert_columns(weather)
    weather = create_date_columns(weather)

    weather.rename(columns={'Date':'date'}, inplace=True)
    # drop some weather and energy price columns for joining
    # weather_dropped = weather.drop(columns=['Year', 'WeekKey', 'day_of_week', 'Week'])

    return weather

In [ ]:
file_path = '/content/drive/MyDrive/master_thesis_data/etmgeg_260 3.txt'
weather = preprocess(file_path)

In [ ]:
weather.to_csv('/content/drive/MyDrive/master_thesis_data/processed_data/weather_processed.csv')

# Renewable energy

In [ ]:
import pandas as pd

def preprocess(ren_energy_path: str) -> pd.DataFrame:
  """
  Ingest and preprocess the renewable energy data.
  """
  ren_energy = pd.read_csv(ren_energy_path, delimiter=';')

  ren_energy = ren_energy[['Perioden', 'Bruto eindverbruik relatief (% van totaal eindverbruik energie)']]
  ren_energy.rename(
    columns={
        'Perioden':'Year',
        'Bruto eindverbruik relatief (% van totaal eindverbruik energie)':'renewableEnergyRatio'
      }, inplace=True)

  ren_energy['Year'] = ren_energy['Year'].str.replace(' **', '')
  ren_energy['renewableEnergyRatio'] = ren_energy['renewableEnergyRatio'].str.replace(',', '.')
  ren_energy['Year'] = ren_energy['Year'].astype(int)
  ren_energy['renewableEnergyRatio'] = ren_energy['renewableEnergyRatio'].astype(float)
  return ren_energy

ren_energy_path = '/content/drive/MyDrive/master_thesis_data/new/Hernieuwbare_energie__verbruik__01062024_121806.csv'
ren_energy = preprocess(ren_energy_path)
ren_energy.to_csv('/content/drive/MyDrive/master_thesis_data/new/preprocessed/renewable_energy.csv')
ren_energy

,Year,renewableEnergyRatio
0,2006,2.78
1,2007,3.30
2,2008,3.60
3,2009,4.27
4,2010,3.92
5,2011,4.53
6,2012,4.66
7,2013,4.69
8,2014,5.42
9,2015,5.73


# Load Consumption

In [ ]:
def preprocess_load_consumption(load_2006_path: str, load_2015_path: str) -> pd.DataFrame:
  # load 2006-2015
  load_2006 = pd.read_excel(load_2006_path, index_col=0)

  load_2006.reset_index(inplace=True)
  load_2006_NL = load_2006[load_2006['Country'] == 'NL']

  df_melted = pd.melt(load_2006_NL, id_vars=["Country", "Year", "Month", "Day", "Coverage ratio"], var_name="Hour", value_name="loadConsumption")
  df_melted.drop(columns=['Country', 'Coverage ratio'], inplace=True)

  df_melted['date'] = pd.to_datetime(df_melted[['Year', 'Month', 'Day']])
  df_melted['datetime'] = pd.to_datetime(df_melted[['Year', 'Month', 'Day', 'Hour']])
  df_melted.drop(columns=['Month', 'Day'], inplace=True)

  # load 2015-2019
  load_2015_2017 = pd.read_excel(load_2015_path, sheet_name='2015-2017')
  load_2018_2019 = pd.read_excel(load_2015_path, sheet_name='2018-2019')

  load = pd.concat([load_2015_2017, load_2018_2019])

  load_NL = load[load['CountryCode'] == 'NL']
  load_NL['DateShort'] = pd.to_datetime(load_NL['DateShort'])
  load_NL = load[load['DateShort'] >= '2016-01-01']
  load_NL.reset_index(drop=True, inplace=True)

  load_NL = load_NL[['DateUTC', 'DateShort', 'Cov_ratio', 'Value', 'Value_ScaleTo100']]

  load_NL['date'] = pd.to_datetime(load_NL['DateShort'], dayfirst=True, format='%d/%m/%Y')
  load_NL['DateUTC'] = pd.to_datetime(load_NL['DateUTC'], format='%d/%m/%Y %H:%M')
  load_NL['Hour'] = load_NL['DateUTC'].dt.hour + 1
  load_NL['Year'] = load_NL['date'].dt.year
  load_NL['Week'] = load_NL['date'].dt.isocalendar().week

  load_NL.rename(columns={'Value': 'loadConsumption', 'DateUTC':'datetime'}, inplace=True)
  load_NL.drop(columns=['Cov_ratio', 'Value_ScaleTo100', 'DateShort', 'Week'], inplace=True)

  load_total = pd.concat([df_melted, load_NL])

  return load_total

load_2006_path = '/content/drive/MyDrive/master_thesis_data/new/load_2006_2015.xlsx'
load_2015_path = '/content/drive/MyDrive/master_thesis_data/new/load_2015_2019.xlsx'


In [ ]:
load_2015_path = '/content/drive/MyDrive/master_thesis_data/new/load_2015_2019.xlsx'
load_2015_2017 = pd.read_excel(load_2015_path, sheet_name='2015-2017')
load_2018_2019 = pd.read_excel(load_2015_path, sheet_name='2018-2019')
load_2018_2019

,MeasureItem,DateUTC,DateShort,TimeFrom,TimeTo,CountryCode,Cov_ratio,Value,Value_ScaleTo100
0,Monthly Hourly Load Values,2018-01-01 00:00:00,2018-01-01,00:00:00,01:00:00,AL,100,746.45,746.45
1,Monthly Hourly Load Values,2018-01-01 00:00:00,2018-01-01,00:00:00,01:00:00,AT,100,6414.00,6414.00
2,Monthly Hourly Load Values,2018-01-01 00:00:00,2018-01-01,00:00:00,01:00:00,BA,100,1249.00,1249.00
3,Monthly Hourly Load Values,2018-01-01 00:00:00,2018-01-01,00:00:00,01:00:00,BE,100,8379.00,8379.00
4,Monthly Hourly Load Values,2018-01-01 00:00:00,2018-01-01,00:00:00,01:00:00,BG,100,3424.49,3424.49
...,...,...,...,...,...,...,...,...,...
476955,Monthly Hourly Load Values,2019-10-31 22:00:00,2019-10-31,22:00:00,23:00:00,HR,100,1767.00,1767.00
476956,Monthly Hourly Load Values,2019-10-31 22:00:00,2019-10-31,22:00:00,23:00:00,HU,100,4804.89,4804.89
476957,Monthly Hourly Load Values,2019-10-31 22:00:00,2019-10-31,22:00:00,23:00:00,MK,100,872.00,872.00
476958,Monthly Hourly Load Values,2019-10-31 22:00:00,2019-10-31,22:00:00,23:00:00,PT,100,5921.00,5921.00


In [ ]:
load = pd.concat([load_2015_2017, load_2018_2019])
load_NL = load[load['CountryCode'] == 'NL']
load_NL['DateShort'] = pd.to_datetime(load_NL['DateShort'])
load_NL = load_NL[load_NL['DateShort'] >= '2016-01-01']
load_NL.reset_index(inplace=True, drop=True)

In [ ]:
load_NL

,MeasureItem,DateUTC,DateShort,TimeFrom,TimeTo,CountryCode,Cov_ratio,Value,Value_ScaleTo100
0,Monthly Hourly Load Values,2016-01-01 00:00:00,2016-01-01,00:00:00,01:00:00,NL,100,11102.0,11102.0
1,Monthly Hourly Load Values,2016-01-01 01:00:00,2016-01-01,01:00:00,02:00:00,NL,100,10657.0,10657.0
2,Monthly Hourly Load Values,2016-01-01 02:00:00,2016-01-01,02:00:00,03:00:00,NL,100,10238.0,10238.0
3,Monthly Hourly Load Values,2016-01-01 03:00:00,2016-01-01,03:00:00,04:00:00,NL,100,10018.0,10018.0
4,Monthly Hourly Load Values,2016-01-01 04:00:00,2016-01-01,04:00:00,05:00:00,NL,100,10017.0,10017.0
...,...,...,...,...,...,...,...,...,...
29177,Monthly Hourly Load Values,2019-04-30 17:00:00,2019-04-30,17:00:00,18:00:00,NL,100,13910.0,13910.0
29178,Monthly Hourly Load Values,2019-04-30 18:00:00,2019-04-30,18:00:00,19:00:00,NL,100,13773.0,13773.0
29179,Monthly Hourly Load Values,2019-04-30 19:00:00,2019-04-30,19:00:00,20:00:00,NL,100,13652.0,13652.0
29180,Monthly Hourly Load Values,2019-04-30 20:00:00,2019-04-30,20:00:00,21:00:00,NL,100,13048.0,13048.0


In [ ]:
# load_NL = load_NL[['DateUTC', 'DateShort', 'Cov_ratio', 'Value', 'Value_ScaleTo100']]

load_NL['date'] = pd.to_datetime(load_NL['DateShort'], dayfirst=True, format='%d/%m/%Y')
load_NL['DateUTC'] = pd.to_datetime(load_NL['DateUTC'], format='%d/%m/%Y %H:%M')
load_NL['Hour'] = load_NL['DateUTC'].dt.hour + 1
load_NL['Year'] = load_NL['date'].dt.year
load_NL['Week'] = load_NL['date'].dt.isocalendar().week

load_NL.rename(columns={'Value': 'loadConsumption', 'DateUTC':'datetime'}, inplace=True)
load_NL.drop(columns=['Cov_ratio', 'Value_ScaleTo100', 'DateShort', 'Week'], inplace=True)


In [ ]:
load_NL

,datetime,loadConsumption,date,Hour,Year
0,2018-01-01 00:00:00,11255.0,2018-01-01,1,2018
1,2018-01-01 01:00:00,10956.0,2018-01-01,2,2018
2,2018-01-01 02:00:00,10647.0,2018-01-01,3,2018
3,2018-01-01 03:00:00,10431.0,2018-01-01,4,2018
4,2018-01-01 04:00:00,10431.0,2018-01-01,5,2018
...,...,...,...,...,...
11633,2019-04-30 17:00:00,13910.0,2019-04-30,18,2019
11634,2019-04-30 18:00:00,13773.0,2019-04-30,19,2019
11635,2019-04-30 19:00:00,13652.0,2019-04-30,20,2019
11636,2019-04-30 20:00:00,13048.0,2019-04-30,21,2019


In [ ]:
load_2006_path = '/content/drive/MyDrive/master_thesis_data/new/load_2006_2015.xlsx'
load_2006 = pd.read_excel(load_2006_path, index_col=0)

load_2006.reset_index(inplace=True)
load_2006_NL = load_2006[load_2006['Country'] == 'NL']

df_melted = pd.melt(load_2006_NL, id_vars=["Country", "Year", "Month", "Day", "Coverage ratio"], var_name="Hour", value_name="loadConsumption")
df_melted.drop(columns=['Country', 'Coverage ratio'], inplace=True)

df_melted['date'] = pd.to_datetime(df_melted[['Year', 'Month', 'Day']])
df_melted['datetime'] = pd.to_datetime(df_melted[['Year', 'Month', 'Day', 'Hour']])
df_melted.drop(columns=['Month', 'Day'], inplace=True)

In [ ]:
df_melted

,Year,Hour,loadConsumption,date,datetime
0,2006,0,10215.0,2006-01-01,2006-01-01 00:00:00
1,2006,0,9590.0,2006-01-02,2006-01-02 00:00:00
2,2006,0,11064.0,2006-01-03,2006-01-03 00:00:00
3,2006,0,11088.0,2006-01-04,2006-01-04 00:00:00
4,2006,0,10925.0,2006-01-05,2006-01-05 00:00:00
...,...,...,...,...,...
87643,2015,23,11567.0,2015-12-27,2015-12-27 23:00:00
87644,2015,23,12135.0,2015-12-28,2015-12-28 23:00:00
87645,2015,23,12225.0,2015-12-29,2015-12-29 23:00:00
87646,2015,23,12205.0,2015-12-30,2015-12-30 23:00:00


In [ ]:
load_total = pd.concat([df_melted, load_NL])
load_total.reset_index(inplace=True, drop=True)

In [ ]:
load_total

,Year,Hour,loadConsumption,date,datetime
0,2006,0,10215.0,2006-01-01,2006-01-01 00:00:00
1,2006,0,9590.0,2006-01-02,2006-01-02 00:00:00
2,2006,0,11064.0,2006-01-03,2006-01-03 00:00:00
3,2006,0,11088.0,2006-01-04,2006-01-04 00:00:00
4,2006,0,10925.0,2006-01-05,2006-01-05 00:00:00
...,...,...,...,...,...
116825,2019,18,13910.0,2019-04-30,2019-04-30 17:00:00
116826,2019,19,13773.0,2019-04-30,2019-04-30 18:00:00
116827,2019,20,13652.0,2019-04-30,2019-04-30 19:00:00
116828,2019,21,13048.0,2019-04-30,2019-04-30 20:00:00


In [ ]:
load_total.to_csv('/content/drive/MyDrive/master_thesis_data/new/preprocessed/load_consumption.csv')


# Geopotential

In [ ]:
import netCDF4 as nc
import pandas as pd
import datetime

path = '/content/drive/MyDrive/master_thesis_data/new/geopotential.nc'
ds = nc.Dataset(path)

geopotential = ds.variables['z']
num_times = geopotential.shape[0]

all_geopotential_values = []

for i in range(num_times):
    geopotential_values = geopotential[i]
    all_geopotential_values.append(geopotential_values.flatten())

df = pd.DataFrame(all_geopotential_values)

df.columns = [f'geopotential{i}' for i in range(df.shape[1])]
df['date'] = [i for i in pd.date_range(start='2006-01-01', end='2019-12-01', freq='MS')]

df.reset_index(inplace=True)

df.drop(columns=['geopotential0',	'geopotential1'], inplace=True)

In [ ]:
geopotential_rows = {}

for i in range(len(df)):
  date = df.iloc[i][-1]
  geopotentials = df.iloc[i][:-1].to_numpy()
  geopotential_rows[date] = geopotentials.reshape(-1, 1)


from sklearn.decomposition import PCA

geopotential_dict = {}

for date, geopotentials in geopotential_rows.items():
  # apply pca to reduce size, we had a lot of values due to multiple longitudes x latitudes
  pca = PCA(n_components=1)
  pca.fit(geopotentials)

  # add to dict
  geopotential_dict[date] = pca.singular_values_[0]

geopotential_df = pd.DataFrame(list(geopotential_dict.items()), columns=['Date', 'geopotential'])

# sst df is missing dates, create range of all dates and merge this to the df
min_date = geopotential_df['Date'].min()
date_range = pd.date_range(start=min_date, end='2023-12-31', freq='D')
df_dates = pd.DataFrame(date_range, columns=['Date'])
df_dates['Date'] = pd.to_datetime(df_dates['Date'])

geopotential_df_all_dates = df_dates.merge(geopotential_df, on = 'Date', how = 'left')

# back fill the empty days
geopotential_df_filled = geopotential_df_all_dates.fillna(method='ffill')

{'TIME': <class 'netCDF4._netCDF4.Variable'>
 float64 TIME(TIME)
     long_name: Time
     units: days since 1800-01-01 00:00:00
     axis: T
     calendar: gregorian
     standard_name: time
 unlimited dimensions: TIME
 current shape = (2221,)
 filling on, default _FillValue of 9.969209968386869e+36 used,
 'NAT': <class 'netCDF4._netCDF4.Variable'>
 float64 NAT(TIME)
     missing_value: -1e+34
     _FillValue: -1e+34
     long_name: NAT Region SSTA (base period 1991-2020, data from OISST)
     units: degC
 unlimited dimensions: TIME
 current shape = (2221,)
 filling on}

In [ ]:
geopotential_df_filled.to_csv('/content/drive/MyDrive/master_thesis_data/processed/geopotential_processed_2006_2009.csv')

# SIC

In [ ]:
import os
import netCDF4 as nc
import pandas as pd
import datetime
from sklearn.decomposition import PCA
import numpy as np
import datetime

In [ ]:
# get paths of sea ice concentration files
sic_paths = [
    '/content/drive/MyDrive/master_thesis_data/new/sic_2006_2007/',
    '/content/drive/MyDrive/master_thesis_data/new/sic_2008_2009/',
    '/content/drive/MyDrive/master_thesis_data/new/sic_2010_2011/',
    '/content/drive/MyDrive/master_thesis_data/new/sic_2012_2013/',
    '/content/drive/MyDrive/master_thesis_data/new/sic_2014_2015/',
    '/content/drive/MyDrive/master_thesis_data/new/sic_2016_2017/',
    '/content/drive/MyDrive/master_thesis_data/new/sic_2018_2019/',
]

# create empty dictionary
sic_dict = {}

for path in sic_paths:
  sea_ice_concentration_files = os.listdir(path)

  # iterate through the files, use try expect because not all files work
  for sic_file in sea_ice_concentration_files:
      try:
          # open file as nc dataset
          ds = nc.Dataset(path + sic_file)

          # get time and ice concentration
          time = ds.variables['time'][:]
          ice_conc = ds.variables['ice_conc'][:]

          # create df from ice concentration and select non nans
          df = pd.DataFrame({'sic': ice_conc.flatten()})
          sic = np.array(df[df['sic'].notna()]['sic']).reshape(-1, 1)

          # apply pca to reduce size, we had a lot of values due to multiple longitudes x latitudes
          pca = PCA(n_components=1)
          pca.fit(sic)

          # add to dict
          sic_dict[time[0]] = pca.singular_values_[0]
      except:
          continue

In [ ]:
# create df from dict
sic_df = pd.DataFrame(list(sic_dict.items()), columns=['time', 'sic'])

# get date from time (time is distance in seconds from 1978-01-01)
sic_df['date'] = sic_df['time'].apply(lambda x: datetime.date(1978, 1, 1) + datetime.timedelta(0, x))
sic_df['date'] = pd.to_datetime(sic_df['date'])

sic_df['year'] = sic_df['date'].dt.year
sic_df['month'] = sic_df['date'].dt.month

sic_df.sort_values('date', inplace=True)
sic_df.reset_index(drop=True, inplace=True)
sic_df = sic_df[['date', 'sic']]

In [ ]:
sic_df.to_csv('/content/drive/MyDrive/master_thesis_data/new/preprocessed/sic_processed.csv')

# Price

In [ ]:
import pandas as pd

price = pd.read_csv('/content/drive/MyDrive/master_thesis_data/new/Eindverbruikersprijzen_01062024_204921.csv', delimiter=';')

price.drop(columns=['Prijscomponenten', 'Belastingen'], inplace=True)

def convert_halfyear_to_date(value):
  year, halfyear, _ = value.split()
  if halfyear == '1e':
      return f"{year}-01-01"
  elif halfyear == '2e':
      return f"{year}-07-01"
  else:
      return None

price['date'] = price['Perioden'].apply(convert_halfyear_to_date)
price.drop(columns=['Perioden'], inplace=True)
price['date'] = pd.to_datetime(price['date'])

for i in price.columns:
  if i != 'date':
    price[i] = price[i].str.replace(',', '.')
    price[i] = price[i].astype(float)

price.rename(columns={
    'Aardgasprijs/Verbruiksklassen huishoudens/Minder dan 569 m3 (euro per m3)':'gasPriceSmall',
    'Aardgasprijs/Verbruiksklassen huishoudens/569 tot 5 687 m3 (euro per m3)':'gasPriceMedium',
    'Elektriciteitsprijs /Verbruiksklassen huishoudens/Minder dan 1 MWh (euro per kWh)':'electricityPriceSmall',
    'Elektriciteitsprijs /Verbruiksklassen huishoudens/1 tot 2,5 MWh (euro per kWh)':'electricityPriceMedium',
    'Elektriciteitsprijs /Verbruiksklassen huishoudens/2,5 tot 5 MWh (euro per kWh)':'electricityPriceLarge',
    'Elektriciteitsprijs /Verbruiksklassen huishoudens/5 tot 15 MWh (euro per kWh)':'electricityPriceXL'
    },
    inplace=True)

min_date, max_date = price['date'].min(), price['date'].max()
date_range = pd.date_range(start=min_date, end=max_date, freq='D')
df_dates = pd.DataFrame(date_range, columns=['date'])


price_all_dates = df_dates.merge(price, on = ['date'], how = 'left')
price_filled = price_all_dates.fillna(method='ffill')

In [ ]:
price_filled['avgGasPrice'] = price_filled[['gasPriceSmall', 'gasPriceMedium']].mean(axis=1)
price_filled['avgElectricityPrice'] = price_filled[['electricityPriceSmall', 'electricityPriceMedium', 'electricityPriceLarge', 'electricityPriceXL']].mean(axis=1)


In [ ]:
price_filled.to_csv('/content/drive/MyDrive/master_thesis_data/new/preprocessed/prices.csv')

# Join


In [ ]:
import pandas as pd

geopotential = pd.read_csv('/content/drive/MyDrive/master_thesis_data/new/preprocessed/geopotential_processed_2006_2019.csv').drop(columns='Unnamed: 0')
sst_df = pd.read_csv('/content/drive/MyDrive/master_thesis_data/processed_data/sst_processed.csv').drop(columns='Unnamed: 0')
energy_prices = pd.read_csv('/content/drive/MyDrive/master_thesis_data/new/preprocessed/prices.csv').drop(columns='Unnamed: 0')
nao = pd.read_csv('/content/drive/MyDrive/master_thesis_data/processed_data/nao_processed.csv').drop(columns='Unnamed: 0')
weather = pd.read_csv('/content/drive/MyDrive/master_thesis_data/processed_data/weather_processed.csv').drop(columns='Unnamed: 0')
sic = pd.read_csv('/content/drive/MyDrive/master_thesis_data/new/preprocessed/sic_processed.csv').drop(columns='Unnamed: 0')
load_consumption = pd.read_csv('/content/drive/MyDrive/master_thesis_data/new/preprocessed/load_consumption.csv').drop(columns='Unnamed: 0')
ren_energy = pd.read_csv('/content/drive/MyDrive/master_thesis_data/new/preprocessed/renewable_energy.csv').drop(columns='Unnamed: 0')


In [ ]:
df = load_consumption.merge(weather, on=['date'] , how='left')
df = df.merge(nao, on=['Year', 'Month'], how='left')
df = df.merge(ren_energy, on = 'Year', how='left')
df = df.merge(energy_prices, on = ['date'], how = 'left')
sst_df.rename(columns={'Date':'date'}, inplace=True)
df = df.merge(sst_df, on = ['date'], how = 'left')
df = df.merge(sic, on = 'date', how = 'left')
geopotential.rename(columns={'Date':'date'}, inplace=True)
df = df.merge(geopotential, on = 'date', how = 'left')

In [ ]:
df.dropna(inplace=True)
df.reset_index(inplace=True, drop=True)

In [ ]:
df

,Year,Hour,loadConsumption,date,datetime,dailyPrecipitation,dailyMeanTemperature,dailyMeanWindspeed,Month,NAO,...,gasPriceMedium,electricityPriceSmall,electricityPriceMedium,electricityPriceLarge,electricityPriceXL,avgGasPrice,avgElectricityPrice,nat,sic,geopotential
0,2009,0,11597.0,2009-01-01,2009-01-01 00:00:00,0.0,-0.6,1.6,1,-0.01,...,0.791,-0.103,0.144,0.198,0.216,0.924,0.11375,0.050574,11967.908197,3.984873e+06
1,2009,0,10774.0,2009-01-02,2009-01-02 00:00:00,0.5,-0.8,2.2,1,-0.01,...,0.791,-0.103,0.144,0.198,0.216,0.924,0.11375,0.050574,12003.010320,3.984873e+06
2,2009,0,11279.0,2009-01-03,2009-01-03 00:00:00,-0.1,-2.7,1.9,1,-0.01,...,0.791,-0.103,0.144,0.198,0.216,0.924,0.11375,0.050574,11979.692685,3.984873e+06
3,2009,0,11204.0,2009-01-04,2009-01-04 00:00:00,4.0,2.3,3.8,1,-0.01,...,0.791,-0.103,0.144,0.198,0.216,0.924,0.11375,0.050574,12022.524093,3.984873e+06
4,2009,0,10776.0,2009-01-05,2009-01-05 00:00:00,2.1,-1.2,5.2,1,-0.01,...,0.791,-0.103,0.144,0.198,0.216,0.924,0.11375,-0.036369,12068.268601,3.984873e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90514,2019,18,13910.0,2019-04-30,2019-04-30 17:00:00,0.0,10.6,2.6,4,0.47,...,0.902,0.078,0.181,0.205,0.211,1.087,0.16875,-0.229531,11710.665035,3.829010e+06
90515,2019,19,13773.0,2019-04-30,2019-04-30 18:00:00,0.0,10.6,2.6,4,0.47,...,0.902,0.078,0.181,0.205,0.211,1.087,0.16875,-0.229531,11710.665035,3.829010e+06
90516,2019,20,13652.0,2019-04-30,2019-04-30 19:00:00,0.0,10.6,2.6,4,0.47,...,0.902,0.078,0.181,0.205,0.211,1.087,0.16875,-0.229531,11710.665035,3.829010e+06
90517,2019,21,13048.0,2019-04-30,2019-04-30 20:00:00,0.0,10.6,2.6,4,0.47,...,0.902,0.078,0.181,0.205,0.211,1.087,0.16875,-0.229531,11710.665035,3.829010e+06


In [ ]:
df.to_csv('/content/drive/MyDrive/master_thesis_data/new/preprocessed/preprocessed_data.csv')